In [1]:
# !pip install MetaTrader5

import MetaTrader5 as mt5
from datetime import datetime, timedelta
import pandas as pd
import time
from tqdm.notebook import tqdm

if not mt5.initialize():
    mt5.shutdown()
mt5.terminal_info()

TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=True, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=False, build=2940, maxbars=100000, codepage=0, ping_last=212076, community_balance=0.0, retransmission=0.0, company='FX Choice Limited', name='FXChoice MetaTrader 5', language='English', path='C:\\Program Files\\FXChoice MetaTrader 5', ...)

In [2]:
data = [_.split('\t') for _ in """

	 ### paste here ####


EURJPY	129.7	0.04	0.202949
EURJPY	133.573	0.04	0.202949
EURUSD	1.17051	0.04	0.0018245
			-
			-
GBPJPY	154.944	0.03	0.2721845
GBPUSD	1.4131	0.03	0.0023825
GBPUSD	1.36883	0.03	0.0023825
USDCAD	1.25059	0.05	0.00212
USDCHF	0.94358	0.06	0.001248
USDCHF	0.84784	0.06	0.001248
USDJPY	110.781	0.07	0.1321125
NZDUSD	0.72412	0.05	0.001505




""".split('\n') if _.split('\t')[0] != '']

data = [
    [
        _[0]
        , float(_[1])
        , round(float(_[2]), 2)
        , round(float(_[3]), 6)
    ] for _ in data
]

print(data)

# for inst in data (unique),
# get current price
# if pending above, sell
# if pending below, buy

p = []
for i in tqdm(set([_[0] for _ in data])):
    a = mt5.symbol_info_tick(i)
    p.append([i, a])

p = [
    [
        _[0], 
        round((_[1].bid + _[1].ask) / 2, 6)
    ] for _ in p
]

print(p)

results = []
for _ in data:
    if _[1] > [x[1] for x in p if x[0] == _[0]][0]:
        print('sell', _, )
        
        r = {
            "action": mt5.TRADE_ACTION_PENDING,
            "symbol": _[0],
            "volume": _[2],
            "type": mt5.ORDER_TYPE_SELL_LIMIT,
            "price": _[1],
            "sl": _[1] + _[3],
            # "tp": _[1] - (_[3]*3),
            "deviation": 10,
            "magic": 9,
            "comment": "sent by python",
            "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(r)  
    else:
        print('buy', _, )
        r = {
            "action": mt5.TRADE_ACTION_PENDING,
            "symbol": _[0],
            "volume": _[2],
            "type": mt5.ORDER_TYPE_BUY_LIMIT,
            "price": _[1],
            "sl": _[1] - _[3],  ############ this is getting mixed up for shorts
            # "tp": _[1] + (_[3]*3),
            "deviation": 10,
            "magic": 9,
            "comment": "sent by python",
            "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(r)  
    results.append(result)

results

[['EURJPY', 129.7, 0.04, 0.202949], ['EURJPY', 133.573, 0.04, 0.202949], ['EURUSD', 1.17051, 0.04, 0.001824], ['GBPJPY', 154.944, 0.03, 0.272184], ['GBPUSD', 1.4131, 0.03, 0.002383], ['GBPUSD', 1.36883, 0.03, 0.002383], ['USDCAD', 1.25059, 0.05, 0.00212], ['USDCHF', 0.94358, 0.06, 0.001248], ['USDCHF', 0.84784, 0.06, 0.001248], ['USDJPY', 110.781, 0.07, 0.132112], ['NZDUSD', 0.72412, 0.05, 0.001505]]



[['EURUSD', 1.187165], ['NZDUSD', 0.694435], ['GBPJPY', 152.1445], ['USDJPY', 110.1825], ['GBPUSD', 1.380835], ['USDCAD', 1.24524], ['EURJPY', 130.806], ['USDCHF', 0.922405]]
buy ['EURJPY', 129.7, 0.04, 0.202949]
sell ['EURJPY', 133.573, 0.04, 0.202949]
buy ['EURUSD', 1.17051, 0.04, 0.001824]
sell ['GBPJPY', 154.944, 0.03, 0.272184]
sell ['GBPUSD', 1.4131, 0.03, 0.002383]
buy ['GBPUSD', 1.36883, 0.03, 0.002383]
sell ['USDCAD', 1.25059, 0.05, 0.00212]
sell ['USDCHF', 0.94358, 0.06, 0.001248]
buy ['USDCHF', 0.84784, 0.06, 0.001248]
sell ['USDJPY', 110.781, 0.07, 0.132112]
sell ['NZDUSD', 0.72412, 0.05, 0.001505]


[OrderSendResult(retcode=10009, deal=0, order=5786771, volume=0.04, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=1, retcode_external=0, request=TradeRequest(action=5, magic=9, order=0, symbol='EURJPY', volume=0.04, price=129.7, stoplimit=0.0, sl=129.497051, tp=130.308847, deviation=10, type=2, type_filling=2, type_time=0, expiration=0, comment='sent by python', position=0, position_by=0)),
 OrderSendResult(retcode=10009, deal=0, order=5786772, volume=0.04, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=2, retcode_external=0, request=TradeRequest(action=5, magic=9, order=0, symbol='EURJPY', volume=0.04, price=133.573, stoplimit=0.0, sl=133.775949, tp=132.964153, deviation=10, type=3, type_filling=2, type_time=0, expiration=0, comment='sent by python', position=0, position_by=0)),
 OrderSendResult(retcode=10009, deal=0, order=5786773, volume=0.04, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=3, retcode_external=0, request

In [16]:
# def send_order(symbol, size, price, risk, side):
    
#     # get current
    
#     # if price > current sell, else buy
    
#     r = {
#         "action": mt5.TRADE_ACTION_PENDING,
#         "symbol": _[0],
#         "volume": _[2],
#         "type": mt5.ORDER_TYPE_SELL_LIMIT,
#         "price": _[1],
#         "sl": _[1] + _[3],
#         "tp": _[1] - (_[3]*3),
#         "deviation": 10,
#         "magic": 9,
#         "comment": "sent by python",
#         "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
#         "type_filling": mt5.ORDER_FILLING_RETURN,
#     }
#     result = mt5.order_send(r) 
    
#     return result

In [3]:
# if comment != "Request executed", then come back to it


In [7]:
results[0]

OrderSendResult(retcode=10021, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='No prices', request_id=7, retcode_external=0, request=TradeRequest(action=5, magic=9, order=0, symbol='CADCHF', volume=0.09, price=0.74104, stoplimit=0.0, sl=0.742109, tp=0.7378330000000001, deviation=10, type=3, type_filling=2, type_time=0, expiration=0, comment='sent by python', position=0, position_by=0))

In [12]:
set([_.request.symbol for _ in results if _.retcode == 10021])

{'CADCHF', 'USDCAD'}

In [8]:
data

[['CADCHF', 0.74104, 0.09, 0.001069],
 ['CADCHF', 0.73336, 0.09, 0.001069],
 ['USDCAD', 1.22602, 0.11, 0.00212],
 ['USDCHF', 0.89361, 0.13, 0.001248]]

In [13]:
# orders to retry
[_ for _ in data if _[0] in set([_.request.symbol for _ in results if _.retcode == 10021])]

[['CADCHF', 0.74104, 0.09, 0.001069],
 ['CADCHF', 0.73336, 0.09, 0.001069],
 ['USDCAD', 1.22602, 0.11, 0.00212]]